In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S1'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 50
Total Tokens Found = 626
Total Tokens Output = 626
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 50
Total Tokens= 626
Unique Tokens= 252
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 50
Sorted Token Size = 252
Clean Token Size = 109
*Stop Replacements here
References Processed =  50
Total Replacement Pairs = 1
Tokens Read = 626
Tokens Changed =  3
References Changed = 3


In [2]:
status="done"

In [3]:
print(status)

done


In [ ]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [4]:
import numpy as np

Create a function of the parameters that returns the F score

In [5]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [6]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 119
NewMatrix Iterateblocks Total Time = 0.37953212670981884


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 71
NewMatrix Iterateblocks Total Time = 0.3649738784879446

Precision = TP / (TP + FP) ......................... = 0.18620690
Recall = TP / (TP + FN) ............................ = 1.00000000
F-Measure .......................................... = 0.31395349


0.31395349

Create functions for variable transformation.

In [7]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [8]:
inv_softplus(1)

0.541324854612918

In [14]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [15]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [9]:
import os, sys

In [10]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [16]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [17]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [18]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-0.7817005703433237

In [19]:
optimizer=None

In [20]:
pbounds = {'beta_sp': (inv_softplus(4), inv_softplus(50)), 
           'sigma_sp': (inv_softplus(4), inv_softplus(50)),
           'mu_logit': (logit(0.5), logit(0.8)),
           'muIncr_logit': (logit(0.01), logit(0.1)),
           'epsilon_sp': (inv_softplus(4), inv_softplus(50)),
           'epsilonIterate_logit': (logit(0), logit(0.25))}

In [21]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
)

In [22]:
optimizer.probe(
    params={"beta_sp": inv_softplus(6), 
            "sigma_sp": inv_softplus(7),
            "mu_logit":logit(0.5),
            'epsilonIterate_logit': logit(0),
            "muIncr_logit":logit(0.1),
            "epsilon_sp":inv_softplus(4.9)},
    lazy=False,
)

Mu:0.50|Mu+:0.10|Beta:6.00|Sigma:7.00|Epsilon:4.90|EpIter:0.00


In [23]:
optimizer.maximize(
    init_points=10,
    n_iter=20,
)

|   iter    |  target   |  beta_sp  | epsilo... | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------------------
Mu:0.55|Mu+:0.02|Beta:23.00|Sigma:8.00|Epsilon:4.01|EpIter:0.00
|  1        |  3.951    |  23.17    | -35.57    |  3.987    | -3.87     |  0.2034   |  8.231    |
Mu:0.64|Mu+:0.04|Beta:12.00|Sigma:35.00|Epsilon:22.24|EpIter:0.00
|  2        |  3.951    |  12.55    | -81.75    |  22.24    | -3.303    |  0.5811   |  35.51    |
Mu:0.64|Mu+:0.05|Beta:13.00|Sigma:29.00|Epsilon:5.25|EpIter:0.00
|  3        |  3.951    |  13.39    | -16.12    |  5.242    | -2.987    |  0.5785   |  29.69    |
Mu:0.61|Mu+:0.09|Beta:10.00|Sigma:35.00|Epsilon:40.83|EpIter:0.00
|  4        |  124.3    |  10.44    | -99.93    |  40.83    | -2.273    |  0.4345   |  35.84    |
Mu:0.56|Mu+:0.01|Beta:44.00|Sigma:44.00|Epsilon:7.90|EpIter:0.00
|  5        |  124.3    |  44.31    | -14.09    |  7.895    | -4.501    |  0.2354   | 

In [24]:
optimizer.max

{'target': 124.33959502167846,
 'params': {'beta_sp': 10.441908843843319,
  'epsilonIterate_logit': -99.92537282598528,
  'epsilon_sp': 40.83056683339417,
  'muIncr_logit': -2.2733299948847447,
  'mu_logit': 0.4344981708207944,
  'sigma_sp': 35.84115276686087}}

In [25]:
inv_logit(optimizer.max["target"])

1.0

In [26]:
params=optimizer.max["params"]

In [27]:
softplus(params["beta_sp"])

10.441938026865925

In [28]:
softplus(params["sigma_sp"])

35.84115276686087

In [29]:
inv_logit(params["mu_logit"])

0.6069472769858874

In [30]:
inv_logit(params["muIncr_logit"])

0.09335597742343671

In [31]:
inv_logit(params["epsilonIterate_logit"])

4.0083162513727074e-44

In [32]:
softplus(params["epsilon_sp"])

40.83056683339417

In [42]:
entropy_F1(mu=0.606,
           muIncr=0.0933,
           beta=10,
           sigma=35,
           epsilon=40.83,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.606
Starting Iteration epsilon= 40.83
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 114
NewMatrix Iterateblocks Total Time = 0.25424656458199024

Precision = TP / (TP + FP) ......................... = 1.00000000
Recall = TP / (TP + FN) ............................ = 1.00000000
F-Measure .......................................... = 1.00000000


1.0